# Parameter tuning for time series forecasting

## ForecastingGridSearchCV


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from aeon.datasets import load_airline
from aeon.utils.plotting import plot_series
from aeon.forecasting.model_selection import temporal_train_test_split
from aeon.forecasting.arima import ARIMA
from aeon.performance_metrics.forecasting import  MeanAbsolutePercentageError, mean_absolute_percentage_error
from aeon.forecasting.model_selection import ExpandingWindowSplitter, ForecastingGridSearchCV

### Setting up

In [ ]:
y = load_airline()
plot_series(y)
plt.show()

In [ ]:
y_train, y_test = temporal_train_test_split(y, test_size=36)

In [ ]:
fh=np.arange(len(y_test)) + 1
print(fh)

### Splitter

In [ ]:
splitter = ExpandingWindowSplitter(fh=fh, initial_window=50, step_length=1)

### Model and parameters

In [ ]:
forecaster = ARIMA(order=(1, 1, 0), seasonal_order=(0, 1, 0, 12))
forecaster.get_params()

In [ ]:
sOrders = np.arange(8, 15)
#
grids = []
for sOrder in sOrders:
    grids.append((1, 1, 0, sOrder))

param_grid = {"seasonal_order": grids}
#
print(param_grid)

### ForecastingGridSearchCV

In [ ]:
gs = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=splitter,
    scoring=MeanAbsolutePercentageError(),
    n_jobs=-1)

### Training

In [ ]:
gs.fit(y_train)

In [ ]:
df = pd.DataFrame(gs.cv_results_)
display(df)

In [ ]:
gs.is_fitted

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

### Test result

In [ ]:
y_pred = gs.predict(fh)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"mape: {mape}")